In [1]:
import tensorflow as tf
print(tf.__version__)
print(tf.keras.__version__)

/Library/Python/3.7/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Library/Python/3.7/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Library/Python/3.7/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Library/Python/3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym

2.0.0-alpha0
2.2.4-tf


/Library/Python/3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Library/Python/3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Library/Python/3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Library/Python/3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWarning: Passing (type

In [2]:
import os
import csv
import time
import datetime
import random
import json
from collections import Counter
from math import sqrt
import gensim
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.keras import backend
from tensorflow.keras.layers import BatchNormalization,Layer,Input,Conv2D,MaxPool2D,concatenate,Flatten,Dense,Dropout,Embedding,Reshape,LSTM
from tensorflow.keras import Sequential,optimizers,losses
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras import regularizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras import initializers
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Flatten
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
from bs4 import BeautifulSoup
import logging
import gensim
from gensim.models import word2vec

import multiprocessing
import yaml
import jieba

In [3]:
class Config(object):
    #数据集路径
    dataSource = "../marriage.txt"
    stopWordSource = "../stopword.txt"
    
    #分词后保留大于等于最低词频的词
    miniFreq=1
    embeddingSize = 200
    
    #统一输入文本序列的定长，取了所有序列长度的均值。超出将被截断，不足则补0
    sequenceLength = 200  
    batchSize=128
    epochs=50
    numClasses = 5
    #训练集的比例
    rate = 0.8  
    dropoutKeepProb=0.5
    ngram_range=2
    max_features = 8000
# 实例化配置参数对象
config = Config()

In [4]:
#中文语料
#设置输出日志
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

file = open("../marriage.txt") 
sentences=[]
with open('../marriage.txt') as fr:
    for line in fr.readlines():
        temp=line.strip().split('\t')
#         sentences.append(jieba.lcut(temp[0]))
        sentences.append(list(temp[0]))

model = word2vec.Word2Vec(sentences,size=config.embeddingSize,
                     min_count=config.miniFreq,
                     window=10,
                     workers=multiprocessing.cpu_count(),sg=1,
                     iter=20)
model.save('../word2VecModel')

In [5]:
model = gensim.models.Word2Vec.load('../word2VecModel')
model.wv.vocab.keys()

dict_keys(['暂', '时', '还', '没', '办', '法', '喽', '你', '是', '哪', '有', '结', '果', '吗', '早', '好', '了', '呀', '都', '一', '年', '多', '嗯', '行', '那', '迷', '魂', '阵', '不', '用', '们', '设', '计', '的', '对', '下', '来', '我', '得', '四', '二', '月', '份', '什', '么', '李', '凤', '啊', '就', '这', '样', '哈', '需', '要', '为', '服', '务', '谢', '因', '已', '经', '红', '颜', '跟', '联', '系', '过', '淘', '宝', '软', '件', '班', '想', '去', '现', '在', '开', '车', '晚', '会', '打', '少', '错', '退', '出', '订', '间', '可', '以', '看', '考', '虑', '些', '问', '题', '后', '再', '说', '吧', '当', '然', '拿', '个', '判', '决', '书', '算', '离', '婚', '证', '前', '像', '听', '到', '文', '肯', '定', '呃', '几', '张', '先', '生', '本', '地', '咯', '哦', '呢', '但', '挖', '掘', '它', '上', '面', '太', '假', '边', '方', '便', '回', '答', '理', '安', '排', '他', '如', '单', '明', '也', '金', '话', '新', '浪', '意', '思', '很', '奇', '弄', '完', '谈', '事', '情', '人', '挺', '麻', '烦', '啦', '六', '家', '账', '号', '叫', '手', '机', '登', '录', '找', '谁', '哎', '担', '心', '怎', '七', '导', '致', '东', '西', '压', '根', '接', '朋', '友', '款', '额', '半', '她', '里', '所', '候',

In [6]:
# 数据预处理的类，生成训练集和测试集
class Dataset(object):
    def __init__(self, config):
        self.dataSource = config.dataSource
        self.stopWordSource = config.stopWordSource  
        # 每条输入的序列处理为定长
        self.sequenceLength = config.sequenceLength  
        self.embeddingSize = config.embeddingSize
        self.rate = config.rate
        self.miniFreq=config.miniFreq
        self.stopWordDict = {}
        self.trainReviews = []
        self.trainLabels = []
        self.evalReviews = []
        self.evalLabels = []
        self.wordEmbedding =None
        self.n_symbols=0
        self.wordToIndex = {}
        self.indexToWord = {}
        
    def readData(self, filePath):
        text=[]
        label=[]
        with open(filePath) as fr:
            for line in file:
                temp=line.strip().split('\t')
                text.append(temp[0])
                label.append(temp[1])
        print('data:',len(text),len(label))
#         texts = [jieba.cut(document) for document in text]
        texts = [list(document) for document in text]
        return texts, label
    
    def readStopWord(self, stopWordPath):
        """
        读取停用词
        """
        with open(stopWordPath, "r") as f:
            stopWords = f.read()
            stopWordList = stopWords.splitlines()
            # 将停用词用列表的形式生成，之后查找停用词时会比较快
            self.stopWordDict = dict(zip(stopWordList, list(range(len(stopWordList)))))
    
    def getWordEmbedding(self, words):
        """
        按照我们的数据集中的单词取出预训练好的word2vec中的词向量
        """
        #中文
        model = gensim.models.Word2Vec.load('../word2VecModel')
        vocab = []
        wordEmbedding = []
        # 添加 "pad" 和 "UNK", 
        vocab.append("pad")
        wordEmbedding.append(np.zeros(self.embeddingSize))
        vocab.append("UNK")
        wordEmbedding.append(np.random.randn(self.embeddingSize))
        for word in words:
            vector =model[word]
            vocab.append(word)
            wordEmbedding.append(vector)           
        return vocab, np.array(wordEmbedding)
    
    def genVocabulary(self, reviews):
        """
        生成词向量和词汇-索引映射字典，可以用全数据集
        """
        allWords = [word for review in reviews for word in review]
        #去掉停用词
        subWords = [word for word in allWords if word not in self.stopWordDict]
        #统计词频，排序
        wordCount = Counter(subWords)  
        sortWordCount = sorted(wordCount.items(), key=lambda x: x[1], reverse=True)
        #去除低频词
        words = [item[0] for item in sortWordCount if item[1] >= self.miniFreq ]
        #获取词列表和顺序对应的预训练权重矩阵
        vocab, wordEmbedding = self.getWordEmbedding(words)
        
        self.wordEmbedding = wordEmbedding
        
        self.wordToIndex = dict(zip(vocab, list(range(len(vocab)))))
        self.indexToWord = dict(zip(list(range(len(vocab))), vocab))
        self.n_symbols = len(self.wordToIndex) + 1
        
        # 将词汇-索引映射表保存为json数据，之后做inference时直接加载来处理数据
        with open("../wordJson/wordToIndex.json", "w", encoding="utf-8") as f:
            json.dump(self.wordToIndex, f)
        with open("../wordJson/indexToWord.json", "w", encoding="utf-8") as f:
            json.dump(self.indexToWord, f)
            
    def reviewProcess(self, review, sequenceLength, wordToIndex):
        """
        将数据集中的每条评论里面的词，根据词表，映射为index表示
        每条评论 用index组成的定长数组来表示
        """
        review=list(review)
        reviewVec = np.zeros((sequenceLength))
        sequenceLen = sequenceLength
        # 判断当前的序列是否小于定义的固定序列长度
        if len(review) < sequenceLength:
            sequenceLen = len(review)
            
        for i in range(sequenceLen):
            if review[i] in wordToIndex:
                reviewVec[i] = wordToIndex[review[i]]
            else:
                reviewVec[i] = wordToIndex["UNK"]
        return reviewVec

    def genTrainEvalData(self, x, y, rate):
        """
        生成训练集和验证集
        """
        reviews = []
        labels = []
        # 遍历所有的文本，将文本中的词转换成index表示
        for i in range(len(x)):
            reviewVec = self.reviewProcess(x[i], self.sequenceLength, self.wordToIndex)
            reviews.append(reviewVec)
            labels.append([y[i]])    
        trainIndex = int(len(x) * rate)
        
        #trainReviews = sequence.pad_sequences(reviews[:trainIndex], maxlen=self.sequenceLength)
        trainReviews = np.asarray(reviews[:trainIndex], dtype="int64")
        trainLabels = np.array(labels[:trainIndex], dtype="float32")
        #evalReviews = sequence.pad_sequences(reviews[trainIndex:], maxlen=self.sequenceLength)
        evalReviews = np.asarray(reviews[trainIndex:], dtype="int64")
        evalLabels = np.array(labels[trainIndex:], dtype="float32")
        return trainReviews, trainLabels, evalReviews, evalLabels
         
    def dataGen(self):
        """
        初始化训练集和验证集
        """
        #读取停用词
        self.readStopWord(self.stopWordSource)
        #读取数据集
        reviews, labels = self.readData(self.dataSource)
        #分词、去停用词
        #生成 词汇-索引 映射表和预训练权重矩阵，并保存
        self.genVocabulary(reviews)
        #初始化训练集和测试集
        trainReviews, trainLabels, evalReviews, evalLabels = self.genTrainEvalData(reviews, labels, self.rate)
        self.trainReviews = trainReviews
        self.trainLabels = trainLabels
        self.evalReviews = evalReviews
        self.evalLabels = evalLabels

data = Dataset(config)
data.dataGen()

data: 6321 6321


/Library/Python/3.7/site-packages/ipykernel_launcher.py:58: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [7]:
def fasttext(config):
    model = Sequential()
    model.add(Embedding(config.max_features, 200, input_length=config.sequenceLength))
    model.add(GlobalAveragePooling1D())
    model.add(Dropout(config.dropoutKeepProb))
    model.add(Dense(config.numClasses, activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

In [8]:
x_train = data.trainReviews
y_train = data.trainLabels
x_test = data.evalReviews
y_test = data.evalLabels

nb_class = y_train.shape[1]
n_symbols=data.n_symbols



# 构建 ngram 数据集
def create_ngram_set(input_list, ngram_value=2):
    """
    Extract a set of n-grams from a list of integers.
    从一个整数列表中提取  n-gram 集合。
    >>> create_ngram_set([1, 4, 9, 4, 1, 4], ngram_value=2)
    {(4, 9), (4, 1), (1, 4), (9, 4)}
    >>> create_ngram_set([1, 4, 9, 4, 1, 4], ngram_value=3)
    [(1, 4, 9), (4, 9, 4), (9, 4, 1), (4, 1, 4)]
    """
    return set(zip(*[input_list[i:] for i in range(ngram_value)]))
 
    
    
def add_ngram(sequences, token_indice, ngram_range=2):
    """
    Augment the input list of list (sequences) by appending n-grams values.
    增广输入列表中的每个序列，添加 n-gram 值
    Example: adding bi-gram
    >>> sequences = [[1, 3, 4, 5], [1, 3, 7, 9, 2]]
    >>> token_indice = {(1, 3): 1337, (9, 2): 42, (4, 5): 2017}
    >>> add_ngram(sequences, token_indice, ngram_range=2)
    [[1, 3, 4, 5, 1337, 2017], [1, 3, 7, 9, 2, 1337, 42]]
    Example: adding tri-gram
    >>> sequences = [[1, 3, 4, 5], [1, 3, 7, 9, 2]]
    >>> token_indice = {(1, 3): 1337, (9, 2): 42, (4, 5): 2017, (7, 9, 2): 2018}
    >>> add_ngram(sequences, token_indice, ngram_range=3)
    [[1, 3, 4, 5, 1337], [1, 3, 7, 9, 2, 1337, 2018]]
    """
    new_sequences = []
    for input_list in sequences:
        new_list = input_list[:]
        for i in range(len(new_list) - ngram_range + 1):
            for ngram_value in range(2, ngram_range + 1):
                ngram = tuple(new_list[i:i + ngram_value])
                if ngram in token_indice:
                    new_list.append(token_indice[ngram])
        new_sequences.append(new_list)
 
    return new_sequences



ngram_range=config.ngram_range
max_features=config.max_features
maxlen=config.sequenceLength


if ngram_range > 1:
    print('Adding {}-gram features'.format(ngram_range))
    # Create set of unique n-gram from the training set.
    
    ngram_set = set()
    for input_list in x_train:
        for i in range(2, ngram_range + 1):
            set_of_ngram = create_ngram_set(input_list, ngram_value=i)
            ngram_set.update(set_of_ngram)
 
    # Dictionary mapping n-gram token to a unique integer. 将 ngram token 映射到独立整数的词典
    # Integer values are greater than max_features in order
    # to avoid collision with existing features.
    # 整数大小比 max_features 要大，按顺序排列，以避免与已存在的特征冲突
    
    start_index = max_features + 1
    token_indice = {v: k + start_index for k, v in enumerate(ngram_set)}
    indice_token = {token_indice[k]: k for k in token_indice}
 
    # max_features is the highest integer that could be found in the dataset.
    # max_features 是可以在数据集中找到的最大的整数
    config.max_features = np.max(list(indice_token.keys())) + 1

    # Augmenting X_train and X_test with n-grams features
    # 使用 n-gram 特征增广 X_train 和 X_test
    x_train = add_ngram(x_train, token_indice, ngram_range)
    x_test = add_ngram(x_test, token_indice, ngram_range)
    print('Average train sequence length: {}'.format(
        np.mean(list(map(len, x_train)), dtype=int)))
    print('Average test sequence length: {}'.format(
        np.mean(list(map(len, x_test)), dtype=int)))
    
    
#填充序列至固定长度
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('X_train shape:', x_train.shape)
print('X_test shape:', x_test.shape)



model = transfromer(config)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=10, mode='auto')
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model_checkpoint = ModelCheckpoint('./fasttext_model/best_model/model_{epoch:02d}-{val_accuracy:.2f}.hdf5', save_best_only=True, save_weights_only=True)
history = model.fit(x_train, y_train, batch_size=config.batchSize, epochs=config.epochs, validation_split=0.3,shuffle=True, callbacks=[reduce_lr,early_stopping,model_checkpoint])



#验证
scores = model.evaluate(x_test, y_test)

#保存模型
yaml_string = model.to_yaml()
with open('./fasttext_model/fasttext_model.yml', 'w') as outfile:
    outfile.write( yaml.dump(yaml_string, default_flow_style=True) )
model.save_weights('./fasttext_model/fasttext_model.h5')

print('test_loss: %f, accuracy: %f' % (scores[0], scores[1]))


Adding 2-gram features


AttributeError: 'numpy.ndarray' object has no attribute 'append'